In [ ]:
# This cell does not get executed when run with Times Square
import os
import datetime

day_obs = int(os.environ.get("SCHEDVIEW_DAY_OBS", datetime.date.today().strftime("%Y%m%d")))
sim_date = datetime.date.fromisoformat(os.environ.get("SCHEDVIEW_SIM_DATE", datetime.date(day_obs//10000, (day_obs%10000)//100, day_obs%100).isoformat()))
sim_index = os.environ.get("SCHEDVIEW_SIM_INDEX", "1")

In [ ]:
# Validate the inputs
import re
assert re.match(r'^\d+$', sim_index) is not None

In [ ]:
import sys
import os

#sched_source = 'env'
sched_source = 'shared'
#sched_source = 'devel'
match sched_source:
    case 'shared':
        if os.path.exists('/sdf/data/rubin/shared/scheduler/packages'):
            sys.path.insert(0, "/sdf/data/rubin/shared/scheduler/packages/uranography-1.2.1")
            sys.path.insert(0, "/sdf/data/rubin/shared/scheduler/packages/rubin_scheduler-3.11.0")
            sys.path.insert(0, "/sdf/data/rubin/shared/scheduler/packages/rubin_sim-2.2.4")
            sys.path.insert(0, "/sdf/data/rubin/shared/scheduler/packages/schedview-0.18.1.dev6+g204b2db")
    case 'devel':
        if os.path.exists('/sdf/data/rubin/user/neilsen/devel'):
            sys.path.insert(0, "/sdf/data/rubin/user/neilsen/devel/uranography")
            sys.path.insert(0, "/sdf/data/rubin/user/neilsen/devel/rubin_scheduler")
            sys.path.insert(0, "/sdf/data/rubin/user/neilsen/devel/rubin_sim")
            sys.path.insert(0, "/sdf/data/rubin/user/neilsen/devel/schedview")
    case _:
        # Use the current environment
        pass

In [ ]:
from schedview.util import config_logging_for_reports
import logging
config_logging_for_reports(logging.ERROR)

In [ ]:
from IPython.display import display, HTML, Markdown
import datetime
import socket
import yaml
from urllib.parse import urlparse
import warnings
import pandas as pd
import numpy as np
import astropy
import bokeh
import bokeh.io
import boto3
import colorcet
from erfa import ErfaWarning
from astropy.time import Time

In [ ]:
display(Markdown(f"# Pre-night briefing report for dayobs {day_obs} with nominal conditions, simulated on {sim_date}"))

In [ ]:
usdf_sim_data_dir = "/sdf/data/rubin/shared/rubin_sim_data"
if os.path.exists(usdf_sim_data_dir):
    os.environ["RUBIN_SIM_DATA_DIR"] = "/sdf/data/rubin/shared/rubin_sim_data"

In [ ]:
import rubin_scheduler
import rubin_scheduler.utils
from rubin_scheduler.scheduler.model_observatory import ModelObservatory
from rubin_sim import maf
from lsst.resources import ResourcePath

In [ ]:
import schedview.compute
import schedview.compute.visits
import schedview.collect
import schedview.collect.rewards
import schedview.plot
import schedview.plot.rewards
from schedview import DayObs

In [ ]:
# Degraded IERS accuracy is never going to be important for these figures.

# If IERS degraded accuracy encountered, don't fail, just keep going.
astropy.utils.iers.conf.iers_degraded_accuracy = "ignore"

In [ ]:
bokeh.io.output_notebook(hide_banner=True)
print(f"bokeh version is {bokeh.__version__}")

In [ ]:
%matplotlib inline

In [ ]:
archive_uri = "s3://rubin:rubin-scheduler-prenight/opsim/"

if urlparse(archive_uri).scheme.upper() == 'S3':
    os.environ["LSST_DISABLE_BUCKET_VALIDATION"] = "1"
    os.environ["S3_ENDPOINT_URL"] = "https://s3dfrgw.slac.stanford.edu/"

In [ ]:
sim_archive_rp = ResourcePath(archive_uri).join(sim_date.isoformat(), forceDirectory=True).join(sim_index, forceDirectory=True)

In [ ]:
sims_found = sim_archive_rp.exists() and sim_archive_rp.join('sim_metadata.yaml').exists()
if not sim_archive_rp.exists():
    display(HTML(f"<b>No simulations found at {sim_archive_rp.geturl()}.</b>"))

if not sim_archive_rp.join('sim_metadata.yaml').exists():
    display(HTML(f"<b>No simulation metadata found at {sim_archive_rp.join('sim_metadata.yaml').geturl()}.</b>"))

In [ ]:
if sims_found:
    sim_archive_metadata = yaml.safe_load(sim_archive_rp.join('sim_metadata.yaml').read().decode())

In [ ]:
if sims_found:
    sim_rp = sim_archive_rp.join(sim_archive_metadata['files']['observations']['name'])

day_obs_mjd = DayObs.from_date(day_obs).mjd
observatory = ModelObservatory(init_load_length=1)
timezone = "Chile/Continental"

## Astronomical events during the night

In [ ]:
day_obs_datetime = Time(day_obs_mjd, format='mjd').datetime
day_obs_date = datetime.date(day_obs_datetime.year, day_obs_datetime.month, day_obs_datetime.day)
night_events = schedview.compute.astro.night_events(day_obs_date)
night_events

## Sun and moon positions in the middle of the night

In [ ]:
model_observatory = ModelObservatory(init_load_length=1)
model_observatory.mjd = night_events.loc['night_middle', 'MJD']

In [ ]:
body_positions_wide = pd.DataFrame(model_observatory.almanac.get_sun_moon_positions(night_events.loc['night_middle', 'MJD']))
body_positions_wide.index.name = 'r'
body_positions_wide.reset_index(inplace=True)

angle_columns = ['RA', 'dec', 'alt', 'az']
all_columns = angle_columns + ['phase']
body_positions = (
    pd.wide_to_long(body_positions_wide, stubnames=('sun', 'moon'), suffix=r'.*', sep='_', i='r', j='')
    .droplevel('r')
    .T[all_columns]
)
body_positions[angle_columns] = np.degrees(body_positions[angle_columns])
body_positions

All angles are in degrees.

## Simulated visits

In [ ]:
if sims_found:
    try:
        visits = schedview.collect.read_opsim(
            sim_rp,
            constraint=f"FLOOR(observationStartMJD-0.5)={day_obs_mjd}",
            stackers=[
                maf.stackers.TeffStacker(),
                maf.stackers.ObservationStartDatetime64Stacker(),
                maf.stackers.ObservationStartTimestampStacker(),
                maf.stackers.DayObsStacker(),
                maf.stackers.DayObsMJDStacker(),
                maf.stackers.DayObsISOStacker(),
                maf.stackers.OverheadStacker()
            ]        
        )
        visits = schedview.compute.visits.add_coords_tuple(visits)
    except UserError:
        visits = None
else:
    visits = None

have_visits = visits is not None and len(visits) > 0

### Numbers of exposures, and gaps between them

In [ ]:
if have_visits:
    overhead_summary = schedview.compute.visits.compute_overhead_summary(visits, night_events.loc['sun_n12_setting','MJD'], night_events.loc['sun_n12_rising','MJD'])
    summary_table = schedview.plot.create_overhead_summary_table(overhead_summary)
    display(HTML(summary_table))
else:
    print("No visits")

### Map of the visits

In [ ]:
if have_visits:
    vmap, vmap_data = schedview.plot.visitmap.create_visit_skymaps(
        visits=visits,
        night_date=day_obs_date,
        timezone=timezone,
        observatory=observatory,
    )
    bokeh.io.show(vmap)
else:
    print("No visits")

The above plots show the visits collected during the night in two different representations, modeled after physical observing tools.

 - The "Armillary sphere" shows the sphere in orthographic projection, with the center point of the projection controlled by the "center alt" and "center az" sliders beneath the plot. A static orthogrophic projection is not an equal-area projection, but playing with the sliders is a helpful way to inform a human's spatial reasoning in three dimensions.
 - The "Planisphere" shows the sky in [Lambert Azimuthal Equal Area Projection](https://en.wikipedia.org/wiki/Lambert_azimuthal_equal-area_projection), centered at the south celestial pole, with R.A. increasing counterclockwise. The projection used is equal area, but highly distorted near the north celestial pole (outside the LSST footprint). This is a particularly helpful representation for planning observing, because changes in time in relevant features are simple rotations, without alterations in distortion, and there are no discontinuities anywhere in the footprint at any time of year.

Both plots show the footprints of camera pointing taken up to the time set by the MJD slider, with the most recent three pointings outlined in cyan. The fill colors are set according to the [descolors palette](https://github.com/DarkEnergySurvey/descolors):

 - <span style='background-color:#56b4e9'>&nbsp;&nbsp;&nbsp;</span><span style='color:#56b4e9'> blue</span>: u band
 - <span style='background-color:#008060'>&nbsp;&nbsp;&nbsp;</span><span style='color:#008060'> green</span>: g band
 - <span style='background-color:#ff4000'>&nbsp;&nbsp;&nbsp;</span><span style='color:#ff4000'> red</span>: r band
 - <span style='background-color:#850000'>&nbsp;&nbsp;&nbsp;</span><span style='color:#850000'> brown/crimson</span>: i band
 - <span style='background-color:#6600cc'>&nbsp;&nbsp;&nbsp;</span><span style='color:#6600cc'> purple</span>: z band
 - <span style='background-color:#000000'>&nbsp;&nbsp;&nbsp;</span><span style='color:#000000'> black</span>: y band

Both plots have the following additional annotations:
 - The gray background shows the planned final depth of the LSST survey.
 - The orange disk shows the coordinates of the moon.
 - The yellow disk shows the coordinates of the sun.
 - The green line (oval) shows the ecliptic.
   - The sun moves along the ecliptic in the direction of increasing R.A. (counter-clockwise in the planisphere figure) such that it makes a full revolution in one year.
   - The moon moves roughly (within 5.14°) along the ecliptic in the direction of increasing R.A. (counter-clockwise in the planisphere figure) , completing a full revolution in one [sidereal month](https://en.wikipedia.org/wiki/Lunar_month#Sidereal_month) (a bit over 27 days), about 14° per day.
 - The blue line (oval) shows the plane of the Milky Way.
 - The black line shows the horizon at the time set by the MJD slider.
 - The red line shows a zenith distince of 70° (airmass=2.9) at the time set by the MJD slider.

### Altitude plot

In [ ]:
if have_visits:
    fig = schedview.plot.nightly.plot_alt_vs_time(
        visits=visits, almanac_events=night_events
    )
    bokeh.io.show(fig)
else:
    print("No visits")

### Horizon plot

In [ ]:
if have_visits:
    fig = schedview.plot.nightly.plot_polar_alt_az(visits=visits, legend=True)
    bokeh.io.show(fig)
else:
    print("No visits")

### Timeline plot

In [ ]:
if have_visits:
    fig = schedview.plot.plot_visit_param_vs_time(visits, 'moonDistance', show_column_selector=True)
    bokeh.io.show(fig)
else:
    print("No visits")

### Rewards by survey

In [ ]:
have_rewards = sims_found and "rewards" in sim_archive_metadata['files']

if have_rewards:
    rewards_rp = sim_archive_rp.join(sim_archive_metadata['files']['rewards']['name'])
    rewards_df, obs_reward = schedview.collect.rewards.read_rewards(rewards_rp)
    rewards_df["queue_fill_mjd_ns"] = rewards_df["queue_fill_mjd_ns"].astype(np.float64)
else:
    rewards_df = None

In [ ]:
if have_rewards:
    plot = schedview.plot.rewards.reward_timeline_for_surveys(rewards_df, day_obs_mjd)
else:
    display(HTML('<div style="color:red; font-size: Large;">No rewards available for this simulation</div>'))

The plot represents the reward (or maximum value of the reward, if the reward is a healpix map rather than a scalar) for each survey with which the scheduler is configured, for each call to the scheduler used to schedule visits on this night.

The horizonatal axis shows the times the scheduler call requested an observation for (not the time at which the call to the scheduler was made).
Each horizontal line shows a timeline of reward values for a given survey.

- Red x's mark infeasible surveys, surveys that cannot be scheduled on the requested time.
- Gray triangles show rewards with an infinite positive value.
- Colored vertical bars represent other values, with limits set by the slider along the bottom of the plot. Both the height and color of the bar are mapped to the value of the reward, and value can also be read by hovering the mouse cursor over the bar.
- Black vertical bars represent rewards that are above the specified color/bar height limit set by the slider (but not infinite), while a red vertical bars represent values below the requested range (but still feasible).

### Rewards by basis function

The following figures show the rewards (or maximum values of the rewards, if the rewards are healpix maps) for each basis function contribution to each survey.

The horizonatal axis shows the times the scheduler call requested an observation for (not the time at which the call to the scheduler was made).
Each horizontal line shows a timeline of reward values for a given survey.

- Red x's mark basis functions which mark a survey as infeasible, such that the survey cannot be scheduled on the requested time.
- Gray triangles show rewards with an infinite positive value.
- Colored vertical bars represent other values, with limits set by the slider along the bottom of the plot. Both the height and color of the bar are mapped to the value of the reward, and value can also be read by hovering the mouse cursor over the bar.
- Black vertical bars represent rewards that are above the specified color/bar height limit set by the slider (but not infinite), while a red vertical bars represent values below the requested range (but still feasible).

In [ ]:
if have_rewards:
    plot = schedview.plot.rewards.nested_tier_reward_timeline_plot(rewards_df, schedview.plot.rewards.reward_timeline_for_tier, day_obs_mjd)
else:
    display(HTML('<div style="color:red; font-size: Large;">No rewards available for this simulation</div>'))

### Feasible area by basis function

The following figures show the area on the sky (in square degrees) for each basis function is feasible..

The horizonatal axis shows the times the scheduler call requested an observation for (not the time at which the call to the scheduler was made).
Each horizontal line shows a timeline of reward values for a given survey.

- Red x's mark basis functions no area on the sky is feasible.
- Blue cirles show basis functions that are feasible over the entire sky, for example those with finite scalar values.
- Colored vertical bars represent other values, with limits set by the slider along the bottom of the plot. Both the height and color of the bar are mapped to the area, and value can also be read by hovering the mouse cursor over the bar.
- Black vertical bars represent rewards that are above the specified color/bar height limit set by the slider (but not infinite), while a red vertical bars represent values below the requested range (but still feasible).

In [ ]:
if have_rewards:
    plot = schedview.plot.rewards.nested_tier_reward_timeline_plot(rewards_df, schedview.plot.rewards.area_timeline_for_tier, day_obs_mjd)
else:
    display(HTML('<div style="color:red; font-size: Large;">No rewards available for this simulation</div>'))

### Table of visits

In [ ]:
# The bokeh table widget and the Rubin RSP notebooks do not work together.
# If running on the Rubin RSP notebook aspect, set on_rsp=True.
# Otherwise, leave it as False
on_rsp = os.getenv("EXTERNAL_INSTANCE_URL", "") == 'https://usdf-rsp.slac.stanford.edu'

In [ ]:
if have_visits:
    for time_column in 'start_timestamp', 'observationStartDatetime64':
        if time_column in visits:
            break
    potential_columns = [time_column, 'fieldRA', 'fieldDec', 'band', 'visitExposureTime', 'numExposures', 't_eff', 'skyBrightness', 'seeingFwhmEff', 'note', 'scheduler_note']
    displayed_columns = [c for c in potential_columns if c in visits]
    if on_rsp:
        with pd.option_context('display.max_rows', 2000):
            display(visits.loc[:,displayed_columns])
    else:
        schedview.plot.create_visit_table(visits, visible_column_names=displayed_columns, width=1024)
else:
    print("No visits")